In [1]:
import os
os.chdir(r"D:\Desktop\MLOPS\ds_wine_prediction")
print("Current Working Directory:", os.getcwd())

Current Working Directory: D:\Desktop\MLOPS\ds_wine_prediction


In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/amisha3k/ds_wine_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="amisha3k"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e454077689b7938f4a4a5f3fc65e4a051bd61360"

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
   

In [4]:
from src.datascience.constants import *
from src.datascience.constants import CONFIG_FILE_PATH
from src.datascience.utils.common import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])   

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        # schema=self.schema.TARGET_COLUMN
        target_column=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
                root_dir=config.root_dir,
                test_data_path=config.test_data_path,
                model_path=config.model_path,
                all_params=params,
                metric_file_name=config.metric_file_name,
                target_column=target_column,
                mlflow_uri="https://dagshub.com/amisha3k/ds_wine_prediction.mlflow"
        )
        return model_evaluation_config

In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import mlflow
import mlflow.sklearn
import joblib
from urllib.parse import urlparse
import urllib.request as request
from src.datascience import logger


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2

    def log_into_mlflow(self):

        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        # mlflow.set_tracking_uri(str(Path(self.config.mlflow_uri).absolute()))
        # #mlflow.set_registry_uri(self.config.mlflow_uri)
        # tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        mlflow_uri = self.config.mlflow_uri
        if mlflow_uri.startswith("http"):
             mlflow.set_tracking_uri(mlflow_uri)   # Remote server (e.g. DagsHub, localhost:5000)
        else:
             mlflow.set_tracking_uri(str(Path(mlflow_uri).absolute()))  # Local folder

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

  
        with mlflow.start_run():

            predicted_qualities=model.predict(test_x)

            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)

            #saving metrics as local
            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_metric("rmse",rmse)   
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
           
            #modelregistry does not work with file store
            if tracking_url_type_store == "file":
            # local file-based MLflow store (works with log_model)
               mlflow.sklearn.log_model(model, "model")
            else:
            # Remote (e.g., DagsHub) → save + log as artifact (no registry)
               mlflow.sklearn.save_model(model, "model")
               mlflow.log_artifacts("model", artifact_path="model")

          

In [11]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
  
except Exception as e:
    raise e 

[2025-09-25 17:38:11,371: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\config\config.yaml]
[2025-09-25 17:38:11,377: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\params.yaml]
[2025-09-25 17:38:11,392: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\schema.yaml]
[2025-09-25 17:38:11,395: INFO: common: Directory created or already exists: artifacts]
[2025-09-25 17:38:11,400: INFO: common: Directory created or already exists: artifacts/model_evaluation]


[2025-09-25 17:38:12,005: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run suave-duck-365 at: https://dagshub.com/amisha3k/ds_wine_prediction.mlflow/#/experiments/0/runs/7195af0af9744e9bb61484c9fcb55e6b
🧪 View experiment at: https://dagshub.com/amisha3k/ds_wine_prediction.mlflow/#/experiments/0
